### Instalação de pacotes

In [1]:
! pip install --upgrade --quiet langchain_community 
! pip install --upgrade --quiet langchain_openai 
! pip install --upgrade --quiet langchain 
! pip install --upgrade --quiet sqlalchemy
! pip install --upgrade --quiet duckdb
! pip install --upgrade --quiet duckdb_engine

### Importações de pacotes

In [2]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase

### O database

O database não é fornecido junto com o laboratório em razão do seu tamanho.<br>
No entanto os dados podem ser obtidos no repositório [datasets](https://github.com/infobarbosa/datasets).

In [3]:
db = SQLDatabase.from_uri("duckdb:///pagamentos.duckdb")

/home/barbosa/.local/lib/python3.10/site-packages/duckdb_engine/__init__.py:173: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [4]:
print(db.dialect)

duckdb


In [5]:
print(db.get_usable_table_names())

['pessoas', 'valores']


In [6]:
db.run("SELECT * FROM pessoas LIMIT 10;")

"[(20128077306, '***.081.588-**', 'ADRIANE DE JESUS SANTOS', 'ITAPIRAPUA PAULISTA', 'SP'), (16358953061, '***.086.753-**', 'MARIA DO ROSARIO GOMES DE MELO', 'MORRO AGUDO', 'SP'), (13622350775, '***.523.498-**', 'ADRIANA SANTOS', 'SAO PAULO', 'SP'), (20732189858, '***.330.168-**', 'DANIELA OLIVEIRA DOS SANTOS', 'SAO PAULO', 'SP'), (20789861660, '***.575.228-**', 'MARCILENE DE SOUZA FERREIRA', 'SAO PAULO', 'SP'), (16076155060, '***.835.265-**', 'TATIANE ALVES SANTOS', 'SAO PAULO', 'SP'), (12102260099, '***.615.358-**', 'MARCIA CRISTINA MONTANHANA', 'CARAGUATATUBA', 'SP'), (20128077306, '***.081.588-**', 'ADRIANE DE JESUS SANTOS', 'ITAPIRAPUA PAULISTA', 'SP'), (16358953061, '***.086.753-**', 'MARIA DO ROSARIO GOMES DE MELO', 'MORRO AGUDO', 'SP'), (20030724885, '***.799.568-**', 'EDNA FERREIRA DOS SANTOS', 'OSASCO', 'SP')]"

### Escolhendo o modelo

In [7]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### Escolhendo a chain

Neste lab vamos usar uma chain que gera consultas SQL com base no prompt especificado. <br>
A documentação pode ser obtida nesse [link](https://api.python.langchain.com/en/latest/chains/langchain.chains.sql_database.query.create_sql_query_chain.html).

In [8]:
chain = create_sql_query_chain(llm, db)

### Enfim o teste!

Vamos começar com algo simples...

In [9]:
sql_text = chain.invoke({"question": "Quantas pessoas tem na tabela pessoas?"})

In [10]:
print(sql_text)

SELECT COUNT("ID_PESSOA") AS total_pessoas FROM pessoas
LIMIT 1;


##### Executando a consulta no `duckdb`

In [11]:
db.run(sql_text)

'[(2458042,)]'

Vamos ver como é o prompt padrão

In [12]:
chain.get_prompts()[0].pretty_print()


You are a DuckDB expert. Given an input question, first create a syntactically correct DuckDB query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per DuckDB. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use today() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of 

##### Outras consultas mais complexas

In [13]:
sql_text = chain.invoke({"question": "Quantas pessoas moram na cidade de são paulo?"})

In [14]:
db.run(sql_text)

'[(662791,)]'

In [15]:
print(sql_text)

SELECT COUNT(*) 
FROM pessoas 
WHERE UPPER("MUNICIPIO") = 'SAO PAULO'


##### Alucinação 1

Curiosamente o modelo não foi capaz de gerar corretamente a próxima consulta.<br>
Ainda estou estudando o porquê...

In [16]:
sql_text = chain.invoke({"question": """
                         Qual o total de pessoas que moram fora da cidade de são paulo e 
                         receberam mais de um pagamento? Me responda com apenas um número."""
                         })

In [17]:
print(sql_text)

SELECT COUNT(DISTINCT p."ID_PESSOA") AS total_pessoas
FROM pessoas p
JOIN valores v ON p."ID_PESSOA" = v."ID_PESSOA"
WHERE p."MUNICIPIO" <> 'SAO PAULO'
GROUP BY p."ID_PESSOA"
HAVING COUNT(v."ID_PAGAMENTO") > 1


In [18]:
db.run(sql_text)

'[(1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1

#### A mesma pergunta, só que diferente...

In [19]:
sql_text = chain.invoke({"question":  
                         """
                         Verifique quantos id_pessoa receberam mais de um pagamento.
                         A seguir verifique quantos desses se referem a pessoas que moram fora da cidade de são paulo. 
                         Me responda com apenas um número.
                         """})

In [20]:
print(sql_text)

SELECT COUNT("ID_PESSOA")
FROM (
    SELECT "ID_PESSOA"
    FROM valores
    GROUP BY "ID_PESSOA"
    HAVING COUNT("ID_PAGAMENTO") > 1
) AS subquery
WHERE "ID_PESSOA" IN (
    SELECT "ID_PESSOA"
    FROM pessoas
    WHERE "MUNICIPIO" <> 'SAO PAULO'
)


Bem melhor!!! ;) <br>
Agora vamos executar.

In [21]:
db.run(sql_text)

'[(2374,)]'

##### Alucinação 2 (regra de negócio!)

Na pergunta a seguir eu peço o nome e total recebido.<br>
Há uma regra de negócio implícita aqui. Consegue perceber qual é? 

In [22]:
sql_text = chain.invoke({"question": 
                         """
                         Busque o nome e total recebido de todas as pessoas que 
                         receberam mais de um pagamento em ordem decrescente.
                         """
                         })

In [23]:
print(sql_text)

SELECT p."NOME", SUM(v."VALOR") AS total_recebido
FROM pessoas p
JOIN valores v ON p."ID_PESSOA" = v."ID_PESSOA"
GROUP BY p."NOME"
HAVING COUNT(v."ID_PAGAMENTO") > 1
ORDER BY total_recebido DESC
LIMIT 5;


O modelo não foi capaz de perceber que é necessário utilizar o id para diferenciar pessoas homônimas.

In [24]:
db.run(sql_text)

"[('MARIA APARECIDA DA SILVA', 921525.0), ('MARIA JOSE DA SILVA', 685751.0), ('ANA PAULA DA SILVA', 503716.0), ('MARIA APARECIDA DOS SANTOS', 492221.0), ('ANA PAULA DOS SANTOS', 302857.0)]"

In [25]:
sql_text = chain.invoke({"question": "Qual a quantidade total de pessoas duplicadas baseado no id? Me responda com um número."})

In [26]:
print(sql_text) 

SELECT COUNT("ID_PESSOA") AS total_duplicadas
FROM (
    SELECT "ID_PESSOA"
    FROM pessoas
    GROUP BY "ID_PESSOA"
    HAVING COUNT("ID_PESSOA") > 1
) AS subquery;


In [27]:
db.run(sql_text)

'[(2965,)]'